In [ ]:
import geopandas as gpd
import networkx as nx
import folium

# Load GeoJSON data
gdf = gpd.read_file('export.geojson')

# Create a NetworkX graph
G = nx.Graph()

# Iterate through the GeoDataFrame
for index, row in gdf.iterrows():
    if 'power' in row and row['power'] == 'tower':
        point = row['geometry']
        coordinates = (point.y, point.x)  # Swap latitude and longitude
        node_id = row['id']
        G.add_node(node_id, pos=coordinates)  # Store coordinates as node attribute

# Create a folium map centered at the mean coordinates
mean_coords = gdf.geometry.unary_union.centroid
m = folium.Map(location=[mean_coords.y, mean_coords.x], zoom_start=10)

# Add markers for power tower nodes
for node_id, attrs in G.nodes(data=True):
    coords = attrs['pos']
    folium.Marker(location=coords, icon=folium.Icon(icon='bolt')).add_to(m)

# Save the map as an HTML file
m.save('power_tower_network.html')

# Display the map in the notebook
m

In [5]:
import geopandas as gpd
import networkx as nx
import folium

from scipy.spatial import distance_matrix
from scipy.sparse.csgraph import minimum_spanning_tree

from sklearn.cluster import DBSCAN

# Load GeoJSON data
gdf = gpd.read_file('export.geojson')

# Create a NetworkX graph
G = nx.Graph()

# Extract tower nodes
tower_nodes = [(row['id'], (row['geometry'].y, row['geometry'].x)) for index, row in gdf.iterrows() if 'power' in row and row['power'] == 'tower']

# Create a list of node positions for distance calculations
positions = [coords for _, coords in tower_nodes]

# Calculate the distance matrix
dist_matrix = distance_matrix(positions, positions)

# Calculate Minimum Spanning Tree
mst = minimum_spanning_tree(dist_matrix).tocoo()

# Add edges to the graph
for i, j, w in zip(mst.row, mst.col, mst.data):
    G.add_edge(tower_nodes[i][0], tower_nodes[j][0], weight=w)

# Cluster with DBSCAN and add new nodes
db = DBSCAN(eps=0.5, min_samples=2).fit(positions)
labels = db.labels_

# Generate new nodes within dense regions (clusters)
new_nodes = []

for label in set(labels):
    if label != -1:
        members = [positions[idx] for idx, _ in enumerate(tower_nodes) if labels[idx] == label]
        centroid = (sum(x[0] for x in members) / len(members), sum(x[1] for x in members) / len(members))
        new_nodes.append(centroid)

# Update the folium map
for edge in G.edges():
    start, end = edge
    locs = [G.nodes[start]['pos'], G.nodes[end]['pos']]
    folium.PolyLine(locs, color="blue", weight=2.5, opacity=1).add_to(m)

for node in new_nodes:
    folium.Marker(location=node, icon=folium.Icon(icon='star')).add_to(m)

# Save the updated map
m.save('power_tower_network_with_links.html')


KeyError: 'pos'